In [1]:
import pandas as pd
import requests
import json

In [2]:
#Link da API para ir buscar os jogos da liga com ID 2021 da season 2024/2025
url = "https://api.football-data.org/v4/competitions/2021/matches?season=2024"

#dados para a conexao com a API
headers = {"X-Auth-Token" : "627885080ca24b25ab7fd22dda93e13c"}

#faz o GET com o URL e com os dados para conexao
response = requests.request("GET", url, headers=headers)

#guarda o resultado da API em formato JSON no data
data = json.loads(response.text)


In [ ]:
print(data)

print(data["matches"])

In [4]:
#cria um dataframe com toda a informação dos jogos, por jogo
df = pd.json_normalize(data["matches"])

In [ ]:
df

In [6]:
#elimina colunas que não são precisas
df2 = df.drop(["status", "stage", "group", "lastUpdated", "referees", "area.id", "area.name", "area.code", "area.flag", "competition.id", "competition.name", "competition.code", "competition.type", "competition.emblem", "season.id", "season.startDate", "season.endDate", "season.currentMatchday", "season.winner","odds.msg", "homeTeam.crest", "awayTeam.crest", "homeTeam.name", "awayTeam.name", "homeTeam.tla","awayTeam.tla","utcDate"], axis = 1)

In [ ]:
df2

In [ ]:
#numero de jogos
N_jogos = df2["matchday"].nunique()
N_jogos

# **começa aqui o tratamento da 1ª parte da questão**

> Adicionar blockquote



In [12]:
#cria um dataframe com as colunas Nome da equipa e ID
Equipas = df2[['homeTeam.shortName', 'homeTeam.id']].drop_duplicates()

In [ ]:
Equipas

In [14]:
#acrescenta ao dataframe anterior a coluna "Pontos_MR" com os valores todos a zero
Equipas["Pontos_MR"] = 0

#garante que a coluna Pontos é INT
Equipas['Pontos_MR'] = Equipas['Pontos_MR'].astype(int)
#garante que a coluna homeTeam.id é INT
Equipas['homeTeam.id'] = Equipas['homeTeam.id'].astype(int)

In [15]:
# Loop pelos jogos
for _, row in df2.iterrows():
    #ve se a equipa que venceu foi a equipa da casa e quem venceu a primeira parte foi a equipa da casa
    if (
        row['score.winner'] == 'HOME_TEAM' and
        row['score.halfTime.home'] > row['score.halfTime.away']
    ):
        #guarda o id da equipa se as condições de cima forem satisfeitas
        id_home = row['homeTeam.id']


        # Verificar se o ID da equipa da casa existe
        if id_home in Equipas['homeTeam.id'].values:
            #acrescenta 1 ponto na coluna Pontos_MR
            Equipas.loc[Equipas['homeTeam.id'] == id_home, 'Pontos_MR'] += 1


In [ ]:
Equipas

In [17]:
#ordena pelo numero de pontos que cada equipa tem
Equipas_SegurarResultadoHOME = Equipas.sort_values(by='Pontos_MR', ascending=False)

In [ ]:
Equipas_SegurarResultadoHOME

In [19]:
#faz a percentagem de vezes que isso aconteceu para cada equipa
Equipas_SegurarResultadoHOME["Percentagem_MR"] = (Equipas_SegurarResultadoHOME["Pontos_MR"] / N_jogos)*100

In [ ]:
Equipas_SegurarResultadoHOME

In [21]:
Equipas_SegurarResultadoHOME = Equipas_SegurarResultadoHOME.rename(columns={"homeTeam.shortName": "Equipa", "homeTeam.id": "ID_Equipa"})

# **começa aqui o tratamento da 2ª questão**

In [ ]:
#cria um dataframe com as colunas Nome da equipa e ID
Equipas_Viraram_Resultado = df2[['homeTeam.shortName', 'homeTeam.id']].drop_duplicates()
Equipas_Viraram_Resultado

In [ ]:
Equipas_Viraram_Resultado["Pontos_VR"] = 0

#garante que a coluna Pontos é INT
Equipas_Viraram_Resultado['Pontos_VR'] = Equipas_Viraram_Resultado['Pontos_VR'].astype(int)
#garante que a coluna homeTeam.id é INT
Equipas_Viraram_Resultado['homeTeam.id'] = Equipas_Viraram_Resultado['homeTeam.id'].astype(int)
Equipas_Viraram_Resultado

In [24]:
# Loop pelos jogos
for _, row in df2.iterrows():
    # Caso a equipa da casa ganhe após estar a perder ao intervalo
    if (
        row['score.winner'] == 'HOME_TEAM'
        and row['score.halfTime.home'] < row['score.halfTime.away']
    ):
        id_home = row['homeTeam.id']

        # Verificar se o ID da equipa da casa existe no DataFrame
        if id_home in Equipas_Viraram_Resultado['homeTeam.id'].values:
            Equipas_Viraram_Resultado.loc[Equipas_Viraram_Resultado['homeTeam.id'] == id_home, 'Pontos_VR'] += 1

    # Caso a equipa visitante ganhe após estar a perder ao intervalo
    elif (
        row["score.winner"] == "AWAY_TEAM"
        and row["score.halfTime.home"] > row["score.halfTime.away"]
    ):
        id_away = row['awayTeam.id']

        # Verificar se o ID da equipa visitante existe no DataFrame
        if id_away in Equipas_Viraram_Resultado['homeTeam.id'].values:
            Equipas_Viraram_Resultado.loc[Equipas_Viraram_Resultado['homeTeam.id'] == id_away, 'Pontos_VR'] += 1


In [ ]:
Equipas_Viraram_Resultado

In [26]:

Equipas_Viraram_Resultado = Equipas_Viraram_Resultado.sort_values(by='Pontos_VR', ascending=False)

In [ ]:
Equipas_Viraram_Resultado

In [28]:
Equipas_Viraram_Resultado["Percentagem_VR"] = (Equipas_Viraram_Resultado["Pontos_VR"] / N_jogos)*100

In [ ]:
Equipas_Viraram_Resultado

In [30]:
Equipas_Viraram_Resultado = Equipas_Viraram_Resultado.rename(columns={"homeTeam.shortName": "Equipa", "homeTeam.id": "ID_Equipa"})

## **Estou a juntar as 2 tabelas numa ós**

In [31]:
# Juntar as tabelas numa só
Equipas_VResultado_MResultado = Equipas_SegurarResultadoHOME.merge(Equipas_Viraram_Resultado[['ID_Equipa', 'Pontos_VR', 'Percentagem_VR']], on='ID_Equipa', how='left')


In [ ]:
Equipas_VResultado_MResultado

## **Vou fazer a classificação final do campionato**

In [ ]:
classificação = df2[['homeTeam.shortName', 'homeTeam.id']].drop_duplicates()
classificação

In [ ]:
classificação["Pontos"] = 0

#garante que a coluna Pontos é INT
classificação['Pontos'] = classificação['Pontos'].astype(int)
#garante que a coluna homeTeam.id é INT
classificação['homeTeam.id'] = classificação['homeTeam.id'].astype(int)
classificação

In [35]:
# Loop pelos jogos
for _, row in df2.iterrows():
    # Caso a equipa da casa ganhe
    if (row['score.winner'] == 'HOME_TEAM'):
        id_home = row['homeTeam.id']

        classificação.loc[classificação['homeTeam.id'] == id_home, 'Pontos'] += 3

    # Caso a equipa visitante ganhe
    elif (row["score.winner"] == "AWAY_TEAM"):
        id_away = row['awayTeam.id']

        classificação.loc[classificação['homeTeam.id'] == id_away, 'Pontos'] += 3

    else:
      id_home = row['homeTeam.id']
      id_away = row['awayTeam.id']

      classificação.loc[classificação['homeTeam.id'] == id_away, 'Pontos'] += 1
      classificação.loc[classificação['homeTeam.id'] == id_home, 'Pontos'] += 1



In [ ]:
classificação

In [37]:
classificação = classificação.sort_values(by='Pontos', ascending=False)

In [ ]:
classificação

In [39]:
classificação = classificação.rename(columns={"homeTeam.shortName": "Equipa", "homeTeam.id": "ID_Equipa"})